# Import Needed Libraries

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Data Preprocessing

### Read data and store it in dataframe

In [2]:
train_data_dic = '/kaggle/input/brain-tumor-mri-dataset/Training'
imagePaths = []
labels =[]

floders = os.listdir(train_data_dic)

for floder in floders:
    floderPath = os.path.join(train_data_dic, floder)
    imageList = os.listdir(floderPath)
    
    for image in imageList:
        imagePath = os.path.join(floderPath, image)
        
        imagePaths.append(imagePath)
        labels.append(floder)
        
        
# Concatenate image paths with labels into one dataframe
Iseries = pd.Series(imagePaths, name= 'imagepaths')
Lseries = pd.Series(labels, name='labels')

train_df = pd.concat([Iseries, Lseries], axis= 1)

In [3]:
train_df

,imagepaths,labels
0,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
1,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
2,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
3,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
4,/kaggle/input/brain-tumor-mri-dataset/Training...,pituitary
...,...,...
5707,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma
5708,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma
5709,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma
5710,/kaggle/input/brain-tumor-mri-dataset/Training...,glioma


In [4]:
test_data_dic = '/kaggle/input/brain-tumor-mri-dataset/Testing'
imagePaths = []
labels =[]

floders = os.listdir(test_data_dic)

for floder in floders:
    floderPath = os.path.join(test_data_dic, floder)
    imageList = os.listdir(floderPath)
    
    for image in imageList:
        imagePath = os.path.join(floderPath, image)
        
        imagePaths.append(imagePath)
        labels.append(floder)
        
        
# Concatenate image paths with labels into one dataframe
Iseries = pd.Series(imagePaths, name= 'imagepaths')
Lseries = pd.Series(labels, name='labels')

ts_df = pd.concat([Iseries, Lseries], axis= 1)

In [5]:
ts_df

,imagepaths,labels
0,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
1,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
2,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
3,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
4,/kaggle/input/brain-tumor-mri-dataset/Testing/...,pituitary
...,...,...
1306,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma
1307,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma
1308,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma
1309,/kaggle/input/brain-tumor-mri-dataset/Testing/...,glioma


### Split dataframe into train, valid, and test

In [6]:
test_df , valid_df = train_test_split(ts_df, test_size=0.5, shuffle=True, random_state=123)

### Create image data generator

In [7]:
img_size = (224, 224)
batch_size = 16

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe(train_df , x_col="imagepaths" , y_col="labels", target_size=img_size , batch_size=batch_size , 
                                       class_mode="categorical", color_mode= 'rgb' , shuffle= True)

valid_gen = ts_gen.flow_from_dataframe(valid_df , x_col="imagepaths" , y_col="labels", target_size=img_size , batch_size=batch_size , 
                                       class_mode="categorical", color_mode= 'rgb' , shuffle= True)

test_gen = ts_gen.flow_from_dataframe(test_df , x_col="imagepaths" , y_col="labels", target_size=img_size , batch_size=batch_size , 
                                       class_mode="categorical", color_mode= 'rgb' , shuffle= True)

Found 5712 validated image filenames belonging to 4 classes.
Found 656 validated image filenames belonging to 4 classes.
Found 655 validated image filenames belonging to 4 classes.


# Model Structure

In [8]:
img_shape = (224, 224 ,3)
class_count = 4

model = Sequential([
    Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu", input_shape=img_shape),
    Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),
    
    Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),
    
    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),
    
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),
    
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(256,activation = "relu"),
    Dense(64,activation = "relu"),
    Dense(class_count, activation = "softmax")

])

model.compile(Adamax(learning_rate=0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,154,180 (80.70 MB)

 Trainable params: 21,154,180 (80.70 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
epochs = 30

history = model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1729993780.529862      65 service.cc:145] XLA service 0x7f1af4002dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729993780.529966      65 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  1/357 ━━━━━━━━━━━━━━━━━━━━ 1:36:45 16s/step - accuracy: 0.0625 - loss: 1.4971

I0000 00:00:1729993792.902395      65 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


357/357 ━━━━━━━━━━━━━━━━━━━━ 60s 124ms/step - accuracy: 0.4488 - loss: 12.9959 - val_accuracy: 0.7317 - val_loss: 0.6697
Epoch 2/30
357/357 ━━━━━━━━━━━━━━━━━━━━ 41s 114ms/step - accuracy: 0.7919 - loss: 0.5263 - val_accuracy: 0.7957 - val_loss: 0.4487
Epoch 3/30
357/357 ━━━━━━━━━━━━━━━━━━━━ 41s 114ms/step - accuracy: 0.8830 - loss: 0.3270 - val_accuracy: 0.8811 - val_loss: 0.2749
Epoch 4/30
357/357 ━━━━━━━━━━━━━━━━━━━━ 41s 114ms/step - accuracy: 0.9152 - loss: 0.2270 - val_accuracy: 0.8872 - val_loss: 0.2953
Epoch 5/30
357/357 ━━━━━━━━━━━━━━━━━━━━ 41s 114ms/step - accuracy: 0.9322 - loss: 0.1862 - val_accuracy: 0.8628 - val_loss: 0.2923
Epoch 6/30
357/357 ━━━━━━━━━━━━━━━━━━━━ 41s 114ms/step - accuracy: 0.9449 - loss: 0.1490 - val_accuracy: 0.8994 - val_loss: 0.3453
Epoch 7/30
357/357 ━━━━━━━━━━━━━━━━━━━━ 41s 114ms/step - accuracy: 0.9576 - loss: 0.1142 - val_accuracy: 0.9207 - val_loss: 0.2308
Epoch 8/30
357/357 ━━━━━━━━━━━━━━━━━━━━ 41s 114ms/step - accuracy: 0.9608 - loss: 0.1058 - va

### Evaluate model

In [10]:
train_score = model.evaluate(train_gen, verbose= 1)
valid_score = model.evaluate(valid_gen, verbose= 1)
test_score = model.evaluate(test_gen, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

357/357 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - accuracy: 1.0000 - loss: 1.7989e-06
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9781 - loss: 0.3240
41/41 ━━━━━━━━━━━━━━━━━━━━ 8s 192ms/step - accuracy: 0.9779 - loss: 0.2359
Train Loss:  1.8239949213239015e-06
Train Accuracy:  1.0
--------------------
Validation Loss:  0.14550496637821198
Validation Accuracy:  0.9832317233085632
--------------------
Test Loss:  0.1791493445634842
Test Accuracy:  0.981679379940033
